In [ ]:
!git config --global user.email "amratanshu99@gmail.com"
!git config --global user.name "Amratanshu Shrivastava"

Cloning into 'StudyBot-Audio-Captioning-and-Q-A-chatbot-'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 81 (delta 23), reused 64 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 129.40 KiB | 1.01 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
!cp /content/audio-captioning.ipynb /content/StudyBot-Audio-Captioning-and-Q-A-chatbot-/

cp: cannot stat '/content/audio-captioning.ipynb': No such file or directory


/content


In [6]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
!pip install -q git+https://github.com/huggingface/peft.git@main

!pip install datasets torchaudio librosa soundfile

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
model_name_or_path = "openai/whisper-large-v2"
task = "transcribe"

In [4]:
language = "English"
language_abbr = "en"

In [ ]:
import os
from datasets import Dataset
import pandas as pd

audio_dir = "dataset/nptel-pure/wav"
text_dir = "dataset/nptel-pure/corrected_txt"

data = []
for fname in sorted(os.listdir(audio_dir)):
    if fname.endswith(".wav"):
        audio_path = os.path.join(audio_dir, fname)
        transcript_path = os.path.join(text_dir, fname.replace(".wav", ".txt"))
        if os.path.exists(transcript_path):
            with open(transcript_path, "r") as f:
                text = f.read().strip()
            data.append({"audio": audio_path, "text": text})

dataset = Dataset.from_pandas(pd.DataFrame(data))

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small")

def prepare(example):
    audio = example["audio"]
    example["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=16000).input_features[0]
    example["labels"] = processor.tokenizer(example["text"]).input_ids
    return example

dataset = dataset.map(prepare, remove_columns=["audio", "text"])

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)